In [ ]:
from pyspark.sql import SparkSession


In [ ]:
spark=SparkSession\
        .builder\
        .appName('ALSExample').config('spark.driver.host', 'localhost')\
        .getOrCreate()


In [ ]:
college_rankings = spark.read.csv('./data/IL_public.csv', header='true', inferSchema='true')

In [ ]:
#check types--drop if weird
college_rankings.dtypes

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.recommendation import ALS

In [ ]:
#train test split
sets(training, test) =college_rankings.randomSplit([0.8, 0.2])

In [ ]:
#coldStart means it will drop nan's
als=ALS(maxIter=5,rank=4, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop')


In [ ]:
predictions=model.transform(test)
evaluator=RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')
rmse=evaluator.evaluate(predictions)
print('Root-mean-square error = '+str(rmse))

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
als_model=ALS(userCol='userId', itemCol='movieId', 
                ratingCol='rating', coldStartStrategy='drop')
#create a parameter grid
params=ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001, 0.1])\
          .addGrid(als_model.rank, [4, 10, 50]).build()
# instantiating crossvalidator estimator
cv=CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model=cv.fit(movie_ratings)    


In [ ]:
#subset users
users=movie_ratings.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs=model.recommendForUserSubset(users, 10)
recs=userSubsetRecs.take(1)


In [ ]:
# use indexing to obtain the movie id of top predicted rated item
first_recommendation=recs[0]['recommendations'][0][0]
# use the name retriever function to get the values
name_retriever(first_recommendation,movie_titles)
'Pirate Radio (2009)'

In [ ]:
recommendations=model.recommendForAllUsers(5)
recommendations.where(recommendations.userId==3).collect()


In [ ]:
defnew_user_recs(user_id, new_ratings, rating_df, movie_title_df, num_recs):
   
 # turn the new_recommendations list into a spark DataFramenew_user_ratings=spark.createDataFrame(new_ratings,rating_df.columns)
    
    # combine the new ratings df with the rating_dfmovie_ratings_combined=rating_df.union(new_user_ratings)
    
    # split the dataframe into a train and test set#     (training, test) = movie_ratings_combined.randomSplit([0.8, 0.2],seed=0)# create an ALS model and fit itals=ALS(maxIter=5,rank=50, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
    model=als.fit(movie_ratings_combined)
    
    # make recommendations for all users using the recommendForAllUsers methodrecommendations=model.recommendForAllUsers(num_recs)
    
    # get recommendations specifically for the new user that has been added to the DataFramerecs_for_user=recommendations.where(recommendations.userId==user_id).take(1)
    
    forranking, (movie_id, rating) inenumerate(recs_for_user[0]['recommendations']):
        movie_string=name_retriever(movie_id,movie_title_df)
        print('Recommendation {}: {}  | predicted score :{}'.format(ranking+1,movie_string,rating))


In [ ]:
user_id=100000user_ratings_1=[(user_id,3253,5),
                  (user_id,2459,5),
                  (user_id,2513,4),
                  (user_id,6502,5),
                  (user_id,1091,5),
                  (user_id,441,4)]
new_user_recs(user_id,
             new_ratings=user_ratings_1,
             rating_df=movie_ratings,
             movie_title_df=movie_titles,
             num_recs=10)
